In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import networkx as nx

import itertools

In [4]:
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)

In [8]:
infilepath = r'/mnt/sda1/Users/Work/SharedSpace_linux/ODs_purpose_mode_timeperiod.csv'

In [9]:
odtemp = pd.read_csv(infilepath, nrows=10)

In [10]:
dtypes_keys = list(odtemp.columns)
dtypes_vals = ['category', 'category', 'category',
               'category', 'category', 'float']

dtypes = dict(zip(dtypes_keys, dtypes_vals))

od = pd.read_csv(infilepath, index_col=list(range(0, 5)))
#od.sort_index(inplace=True)

In [11]:
od.head(2)

Trips
orig_zoneID dest_zoneID Purpose Mode TimePeriod           
E02000001   E02000001   HBO     Bus  AM          20.572919
            E02000576   HBO     Bus  AM           3.849908

In [12]:
d_grp_purp = {'HBW': 'HB', 'HBO': 'HB', 'NHB': 'NHB'}
grp_keys = [None, None, d_grp_purp, None, None]

odg = od.groupby(grp_keys, level=od.index.names).sum()

In [13]:
odg.reset_index(inplace=True)
odg.Purpose = odg.Purpose.astype('category')

In [14]:
odg.head(2)

,orig_zoneID,dest_zoneID,Purpose,Mode,TimePeriod,Trips
0,E02000001,E02000001,HB,Bus,AM,93.610267
1,E02000001,E02000001,HB,Bus,IP,189.761532


for c in odg:
    if odg[c].dtype.name == 'category':
        odg[c] = odg[c].cat.codes

In [15]:
odg.set_index(['orig_zoneID', 'dest_zoneID', 'Purpose', 'Mode', 'TimePeriod'], inplace=True)

In [16]:
od_car = odg.xs('CarDriver', level='Mode')

In [17]:
od_car.reset_index(inplace=True)

for c in od_car:
    if od_car[c].dtype.name == 'category':
        od_car[c] = od_car[c].cat.codes

In [77]:
# Reduce
od_car_reduced = od_car[od_car.Trips >= 50]
# Remove intrazonals
od_car_reduced = od_car_reduced[od_car_reduced.orig_zoneID != od_car_reduced.dest_zoneID]

In [78]:
G = nx.from_pandas_dataframe(od_car_reduced, 'orig_zoneID', 'dest_zoneID',
                             ['Purpose', 'TimePeriod'], nx.MultiDiGraph())

In [79]:
G.number_of_nodes()

558

In [80]:
G.number_of_edges()

6168

In [103]:
potential_tripchains = nx.trip_chains(G, 5)

In [84]:
def viable_tripchains(G, tripchains):
    
    def filter_edges_purpose(start_node, end_node, purpose):
        edges_data = G.get_edge_data(start_node, end_node)
        filt_edges = [v for k, v in edges_data.items() if v['Purpose'] == purpose]
        
        if filt_edges:
            return filt_edges
        else:
            return None
        
    
    def filter_interm_edges(interm_edges):
        filt_interm_edges = []
        for u, v in pairwise(interm_edges):
            filt_edges = filter_edges_purpose(u, v, 'NHB')

            if filt_edges:
                filt_interm_edges.append(filt_edges)
            else:
                return None
        
        return filt_interm_edges

    # Filter edges 
    # Filter intrazonals
    
    viable_tripchains = []
    
    for tc in tripchains:
        # We filter the edges of the tripchain according to purpose
        edges = []
        if len(tc) > 1:
            
            first_edges = filter_edges_purpose(tc[0], tc[1], 'HB')
            last_edges = filter_edges_purpose(tc[-2], tc[-1], 'HB')
            
            # Check if first and last leg fulfil the criteria
            # If they don't function has returned NaN
            if first_edges and last_edges:
                edges.append(first_edges)
                edges.append(last_edges)
            else:
                continue
        
            if len(tc) > 2:
                # The rest can only be NHB trips
                interm_edges = tc[1:-1]
                
                filt_interm_edges = filter_interm_edges(interm_edges)    
                
                if filt_interm_edges:
                    edges.insert(1, interm_edges)
                else:
                    continue
            
            viable_tripchains.append(edges)

In [95]:
first_leg_edges_data

dict_values([{'Purpose': 'HB', 'TimePeriod': 'AM'}, {'Purpose': 'HB', 'TimePeriod': 'IP'}, {'Purpose': 'HB', 'TimePeriod': 'OP'}, {'Purpose': 'HB', 'TimePeriod': 'PM'}, {'Purpose': 'NHB', 'TimePeriod': 'IP'}])

In [74]:
potential_tripchains = nx.trip_chains(G, 5)

%time sum(1 for _ in potential_cycles)

In [91]:
tc = next(potential_tripchains)
tc

['E02000231', 'E02000221', 'E02000217']

In [92]:
G.get_edge_data(tc[0], tc[1])

{0: {'Purpose': 'HB', 'TimePeriod': 'IP'}}

In [93]:
G.get_edge_data(tc[1], tc[2])

{0: {'Purpose': 'HB', 'TimePeriod': 'AM'},
 1: {'Purpose': 'HB', 'TimePeriod': 'IP'},
 2: {'Purpose': 'HB', 'TimePeriod': 'OP'},
 3: {'Purpose': 'HB', 'TimePeriod': 'PM'},
 4: {'Purpose': 'NHB', 'TimePeriod': 'IP'}}

In [68]:
G.get_edge_data(pc[2], pc[3])

IndexError: list index out of range

In [95]:
df = od_car_red.copy()

In [100]:
dft = df.set_index(list(df.columns[:-1])).xs('NHB', level='Purpose')